In [ ]:
from itertools import chain
from pathlib import Path

from skrt import Patient
from skrt.dose import remove_duplicate_doses

# Define paths to patient data.
data_dir = Path("~/data/20220331_import_data_selection").expanduser()
paths = sorted(list(chain(data_dir.glob('import_high/H*'),
                          data_dir.glob('import_low/L*'))))

In [ ]:
# Load data, and calculate dose sums.
patients = []
for path in paths[0:]:
    patients.append(Patient(path, unsorted_dicom=True))
    patients[-1].get_dose_sum()
    print(f"{patients[-1].id} - initialisation; {patients[-1]._init_time:.2f} s; "
          f"dose sum: {patients[-1]._dose_sum_time:.2f} s")

In [ ]:
def get_structure_sets(patient, clinical_rois=False):
    """
    Retrieve structure sets for planning and relapse scan.
    
    **Parameter:**
    
    patient : skrt.patient.Patient
        Patient object for which structure sets are to be retrieved.
        
    clinical_rois : bool, default=False
        If True, create ROI masks for clinical structure set,
        as well as for (research) plan and relapse structure sets.
        This can be quite time consuming, so isn't done by default.
    """
    # Obtain structure sets, sorted by number of ROIs.
    structure_sets = sorted(patient.combined_objs("structure_set_types"),
                            key=(lambda ss : len(ss.get_roi_names())))

    # Identify the control structures, which should always be outlined in the research structure sets.
    control_rois = [["Carina"], ["Spinal canal", "Spinal Canal", "Spinal cord"], ["Top of sternum", "Top of Sternum"]]
    
    # Remove from list of structure sets the cases where the number of ROIs saved
    # is less than the number of control structures.
    while len(structure_sets[0].get_roi_names()) < len(control_rois):            
        structure_sets.pop(0)
        
    # The plan research structure set should be the one with fewest ROIs.
    ss_plan = structure_sets[0]
    assert len(ss_plan.get_roi_names()) == len(control_rois)
    for names in control_rois:
        assert [name in ss_plan.get_roi_names() for name in names].count(True) == 1

    # Remove from list of structure sets the cases where the number of ROIs saved
    # is equal to the number of control structures.
    while len(structure_sets[0].get_roi_names()) == len(control_rois):
        structure_sets.pop(0)
        
    # Among the remaining structure sets, the relapse research structure set should
    # include the control structures and at least one recurrence outline.
    # Among the structure sets that don't include the control structures, the clinical
    # structure set should be the largest.
    ss_relapse = ss_clinical = None
    for structure_set in structure_sets:
        has_control_rois = True
        for names in control_rois:
            has_control_rois *= ([name in structure_set.get_roi_names() for name in names].count(True) == 1)
        if has_control_rois:
            if (not ss_relapse) or (len(ss_relapse.get_roi_names()) > len(structure_set.get_roi_names())):
                ss_relapse = structure_set
        else:
            if (not ss_clinical) or (len(ss_clinical.get_roi_names()) > len(structure_set.get_roi_names())):
                ss_clinical = structure_set
            
    # Set image for plan structure set, and optionally for clinical structure set,
    # to be the image associated with the dose sum.
    ss_plan.set_image(patient.get_dose_sum().image)
    if clinical_rois:
        ss_clinical.set_image(patient.get_dose_sum().image)
    
    # Set image for relapse structure set to be the first (only?) image that has at least 20 slices,
    # and doesn't have the same number of slices as the image associated with the dose.
    # (For this sample, images for a patient that have the same number of slices are the same image,
    # potentially resampled.)
    for image in patient.combined_objs("image_types"):
        if len(image.dicom_paths) >= 20 and image.get_n_voxels()[2] != patient.get_dose_sum().image.get_n_voxels()[2]:
            ss_relapse.set_image(image)
            break
    
    return(ss_plan, ss_relapse, ss_clinical)

In [ ]:
# Display dose sums, overlaid on ct scans.
no_ui = False
view = "x-y"
for patient in patients:
    dose_sum = patient.get_dose_sum()
    print(f"\n{patient.id} - maximum dose: {dose_sum.get_max():.2f} Gy "
          f"{[dose.get_dose_summation_type() for dose in remove_duplicate_doses(patient.combined_objs('dose_types'))]}")
    vmax = 60 if patient.id.startswith("L") else 60
    ss_plan, ss_relapse, ss_clinical = get_structure_sets(patient)
    print(f"Plan ROIs: {ss_plan.get_roi_names()}")
    print(f"Relapse ROIs: {ss_relapse.get_roi_names()}")
    print(f"Clinical ROIs: {ss_clinical.get_roi_names()}")
"""
    dose_sum.image.view(images=(ss_relapse.image if ss_relapse else None), init_view=view,
                        annotate_slice="yellow",
                        init_idx=dose_sum.get_centroid_idx(view, 0.9),
                        dose=dose_sum, intensity=(-300, 100), colorbar=-1,
                        colorbar_last_only=False, ytick_labels_first_only=False,
                        dose_kwargs={"vmin": 0, "vmax": vmax},
                        rois=[ss_clinical, None], legend=False, no_ui=no_ui)
"""